In [1]:
%%capture
# !pip install -U --force-reinstall sierra-ils-utils

# install from this branch for testing
# !pip install -U --force-reinstall git+https://github.com/chimpy-me/sierra-ils-utils.git@model/QueryEntry
# !pip install -U --force-reinstall git+https://github.com/chimpy-me/sierra-ils-utils.git

In [2]:
# !pip freeze | grep sierra-ils-utils

In [3]:
# import the modules
from sierra_ils_utils import SierraAPI, QueryBuilder
# from google.colab import userdata
import json
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

In [4]:
# from google.colab import userdata

with open('./.config.json') as f:
  config = json.load(f)

sierra_api = SierraAPI(
    # sierra_api_base_url=userdata.get('sierra_api_base_url'),
    sierra_api_base_url=config.get('sierra_api_base_url'),
    # sierra_api_key=userdata.get('sierra_api_key'),
    sierra_api_key=config.get('sierra_api_key'),
    # sierra_api_secret=userdata.get('sierra_api_secret'),
    sierra_api_secret=config.get('sierra_api_secret'),
    log_level=logging.DEBUG
)


# # setup the api client from sierra-ils-utils
# sierra_api = SierraAPI(
#     sierra_api_base_url=userdata.get('sierra-test_api_base_url'),
#     sierra_api_key=userdata.get('sierra-test_api_key'),
#     sierra_api_secret=userdata.get('sierra-test_api_secret'),
#     log_level='DEBUG'
# )

DEBUG:sierra_ils_utils.sierra_ils_utils:INIT {'base_url': 'https://classic.cincinnatilibrary.org/iii/sierra-api/v6/', 'api_key': '1ebmZG6N********************', 'request_count': 0, 'expires_at': 0, 'session_headers': {'accept': 'application/json', 'Authorization': ''}, 'endpoints': {'GET': {'bibs/': {'responses': {200: <class 'sierra_ils_utils.sierra_api_v6_endpoints.BibResultSet'>, 400: <class 'sierra_ils_utils.sierra_api_v6_endpoints.ErrorCode'>, 404: <class 'sierra_ils_utils.sierra_api_v6_endpoints.ErrorCode'>}, 'response_model': <class 'sierra_ils_utils.sierra_api_v6_endpoints.BibResultSet'>}, 'bibs/{id}': {'responses': {200: <class 'sierra_ils_utils.sierra_api_v6_endpoints.Bib'>, 400: <class 'sierra_ils_utils.sierra_api_v6_endpoints.ErrorCode'>, 404: <class 'sierra_ils_utils.sierra_api_v6_endpoints.ErrorCode'>}, 'response_model': <class 'sierra_ils_utils.sierra_api_v6_endpoints.Bib'>}, 'info/token': {'responses': {200: <class 'sierra_ils_utils.sierra_api_v6_endpoints.TokenInfo'>},

In [5]:
# we have a list of 100 item barcodes in a .csv file ...
# ... we're going to load them into a list

import csv

with open('./current_collection-a9be2bf.csv') as f:
    reader = csv.reader(f)             # use the csv reader to read the file
    columns = next(reader)             # first "row" is the column names
    rows = [row[0] for row in reader]  # produce a list of barcodes

print(
    columns,   # column names
    rows[:3],  # first 3 barcodes
    len(rows), # number of barcodes
    sep='\n'
)

['barcode']
['0923112556017', 'A000074528052', '8264518556018']
100


In [6]:
# sierra-ils-utils also has a (simple) query builder

# https://documentation.iii.com/sierrahelp/#sgil/sgil_lists_json.html
# > You can view and define Create Lists queries using JavaScript Object
# > Notation (JSON)

query = QueryBuilder()  # instatiate the class

# the query builder allows for "method chaining":
# ... below, we start a query with a target type of "item", and varfield tag "b"
# ... followed by the barcodes, "b" in the list of operands
# e.g. item records having a matching barcodes to ones in our list
query = query.start_query(record_type='item', field_tag='b') \
    .add_expression(op='in', operands=rows) \
    .end_query()  # end the query

note = """
this generates the following similar JSON by using the method, `query.json()`

{
  "target": {
    "record": {
      "type": "item"
    },
    "field": {
      "tag": "b"
    }
  },
  "expr": [
    {
      "op": "in",
      "operands": [
        "0923112556017",
        "A000074528052",
        "8264518556018"
      ]
    }
  ]
}
"""

In [7]:
# send the query!
response = sierra_api.post(
    'items/query',
    params={
        'offset': 0,
        'limit': 2000,
    },
    json_body=query.json()  # send the query as JSON
)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): classic.cincinnatilibrary.org:443
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "POST /iii/sierra-api/v6/token HTTP/1.1" 200 230
INFO:sierra_ils_utils.sierra_ils_utils:Authorization Success. response.json.get('expires_in'): 3600
INFO:sierra_ils_utils.sierra_ils_utils:Sierra session authenticated
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/info/token HTTP/1.1" 200 3584
INFO:sierra_ils_utils.sierra_ils_utils:Sierra response status code                  : 200
INFO:sierra_ils_utils.sierra_ils_utils:Sierra 'expiresIn'                           : 3599
INFO:sierra_ils_utils.sierra_ils_utils:session expires at (UNIX Epoch)              : 1701576822.7345364
INFO:sierra_ils_utils.sierra_ils_utils:seconds left                                 : 3539.939050912857
INFO:sierra_ils_utils.sierra_ils_utils:request url                                  : https://class

In [8]:
print(
    len(response.data.entries),  # number of entries
    response.data.entries[:3],    # print the first 3 entries
    sep='\n'
)

100
[QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/1008510'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/1051453'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/1089999')]


In [20]:
# now that we have the items record numbers, we can get item data

result = sierra_api.get(
    'items/',
    params={
        'id': ','.join(response.data.entry_ids),
        'fields': 'default'
    }
)

DEBUG:sierra_ils_utils.sierra_ils_utils:GET {"endpoint": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/", "params": "{'id': '1008510,1051453,1089999,1128285,1149989,1310822,1601060,1793192,1968380,2006419,2020016,2020707,2030791,2056420,2093246,2268916,2315163,2320049,2374433,2599992,2638456,2665460,2860311,2991150,3113983,3144330,3148531,3191696,3247921,3727599,3791673,3831990,3956454,3975242,4060167,4196464,4254422,4387071,4782511,4799691,4897638,4911080,5174099,5280541,5293996,5465619,5552068,5579548,5626824,5729432,5776816,5949912,7761942,7871518,8101516,8208741,8423080,8603237,8605599,8677901,8736267,8980877,9059912,9222526,9250700,9451597,9539229,9677344,9678922,9943807,9960593,10135506,10238009,10265809,10401599,10420145,10421589,10456329,10616213,10669667,10779898,10938336,11035487,11140377,11291644,11346531,11461660,11496797,11503993,11506262,11540323,11622305,11742854,11824107,11860845,11896081,11955479,12104387,12108965,12161411', 'fields': 'default', 'limit

In [29]:
# result.data.entries[0]

# if items have a status->duedate, or volumes, don't place a hold on it.
items_for_holds = []
items_not_for_holds = []

for item in result.data.entries:
    if len(item.volumes) == 0 and item.status.duedate is None:
        # print(f'{item.id} PUT HOLD\n')
        items_for_holds.append(item.id)
    else:
        # print(f'{item.id} DON\'T PUT HOLD\n')
        items_not_for_holds.append(item.id)

1008510 PUT HOLD

1051453 DON'T PUT HOLD

1089999 DON'T PUT HOLD

1128285 PUT HOLD

1149989 PUT HOLD

1310822 PUT HOLD

1601060 PUT HOLD

1793192 PUT HOLD

1968380 PUT HOLD

2006419 PUT HOLD

2020016 PUT HOLD

2020707 PUT HOLD

2030791 DON'T PUT HOLD

2056420 PUT HOLD

2093246 PUT HOLD

2268916 DON'T PUT HOLD

2315163 PUT HOLD

2320049 PUT HOLD

2374433 PUT HOLD

2599992 PUT HOLD

2638456 DON'T PUT HOLD

2665460 PUT HOLD

2860311 DON'T PUT HOLD

2991150 DON'T PUT HOLD

3113983 PUT HOLD

3144330 PUT HOLD

3148531 PUT HOLD

3191696 PUT HOLD

3247921 PUT HOLD

3727599 PUT HOLD

3791673 PUT HOLD

3831990 PUT HOLD

3956454 PUT HOLD

3975242 PUT HOLD

4060167 PUT HOLD

4196464 PUT HOLD

4254422 PUT HOLD

4387071 PUT HOLD

4782511 PUT HOLD

4799691 PUT HOLD

4897638 PUT HOLD

4911080 DON'T PUT HOLD

5174099 PUT HOLD

5280541 PUT HOLD

5293996 PUT HOLD

5465619 PUT HOLD

5552068 PUT HOLD

5579548 PUT HOLD

5626824 DON'T PUT HOLD

5729432 PUT HOLD

5776816 PUT HOLD

5949912 DON'T PUT HOLD

7761

In [24]:
item.status.duedate

'2023-12-04T09:00:00Z'

In [ ]:
# the create lists json query ...
json_body = """\
{
  "target": {
    "record": {"type": "bib"},
    "field": {"tag": "t"}
  },
  "expr": {
    "op": "has",
    "operands": ["spider-man"]
  }
}
"""

response = sierra_api.post(
    'bibs/query',
    params={
        'offset': 0,
        'limit': 10,
    },
    json_body=json_body
)

print(response)

In [ ]:
for entry in response.data.entry_ids:
  print(entry)

In [ ]:
print(response.data.entry_ids)

In [ ]:
if response.data.total and response.data.total > 0:
    for i, entry in enumerate(response.data.entries, start=1):
        print(i, entry.link)
        bib_response = sierra_api.get(
            'bibs/{id}',
            params={
                'fields': 'title'
            },
            path_params={
                'id': entry.link.split('/')[-1]  # just need the id or record num portion
            }
        )

        print(bib_response.data.title)